<a href="https://colab.research.google.com/github/vfrantc/dehaze_detect/blob/main/evaluate_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


# RUN DETECTOR ON UNPROCESSED IMAGES

In [ ]:
!pip install mmcv-full
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -e .
!pip install Pillow==7.0.0

     |████████████████████████████████| 389 kB 7.6 MB/s 
     |████████████████████████████████| 185 kB 58.8 MB/s 


In [ ]:
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmdetection/v2.0/scnet/scnet_x101_64x4d_fpn_20e_coco/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth \
      -O checkpoints/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth

In [ ]:
from glob import glob
import os

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.datasets import build_dataloader
from mmdet.apis import single_gpu_test
from mmcv.parallel import MMDataParallel
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

config = 'configs/scnet/scnet_x101_64x4d_fpn_20e_coco.py'
checkpoint = 'checkpoints/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth'
model = init_detector(config, checkpoint, device='cuda:0')
model = MMDataParallel(model, device_ids=[0])

Use load_from_local loader


In [ ]:
def create_dataset(folder):
  cfg = Config.fromfile('./configs/scnet/scnet_x101_64x4d_fpn_20e_coco.py')
  cfg.dataset_type = 'CocoDataset'
  PREFIX = folder
  cfg.data.test.img_prefix = folder
  cfg.data.test.ann_file = os.path.join(PREFIX, 'new_test.json')
  cfg.data.test.type = 'CocoDataset'
  dataset = build_dataset(cfg.data.test)
  #dataset.CLASSES = classes
  #dataset.cat_ids = list(range(len(dataset.CLASSES)))
  data_loader = build_dataloader(dataset, samples_per_gpu=1, workers_per_gpu=2, dist=False, shuffle=False)
  return dataset, data_loader

In [ ]:
!cp /content/drive/MyDrive/hazy_detect/out_gca.zip .
!cp /content/drive/MyDrive/hazy_detect/dcp_coco.zip . 
!cp /content/drive/MyDrive/hazy_detect/out_refined.zip .
!cp /content/drive/MyDrive/hazy_detect/dehazenet_coco.zip .
!cp /content/drive/MyDrive/hazy_detect/ide_coco.zip .
!cp /content/drive/MyDrive/hazy_detect/rtts_coco.zip .

In [ ]:
!unzip -q out_gca.zip
!unzip -q dcp_coco.zip
!unzip -q out_refined.zip
!unzip -q dehazenet_coco.zip
!unzip -q ide_coco.zip
!unzip -q rtts_coco.zip

### gca

In [ ]:
method = 'out_gca'
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
outputs = single_gpu_test(model, data_loader, True, f'./out_{method}', 0.3)
dataset.cat_ids.append(91)
metric = dataset.evaluate(outputs, metric='bbox')

!zip -r detected_gca.zip out_out_gca/
!cp -r detected_gca.zip /content/drive/MyDrive/hazy_detect

loading annotations into memory...
Done (t=0.29s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 2.4 task/s, elapsed: 1836s, ETA:     0s

IndexError: ignored

### dcp


In [ ]:
method = 'dcp_coco'
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, True, f'./out_{method}', 0.3)
metric = dataset.evaluate(outputs, metric='bbox')

loading annotations into memory...
Done (t=0.15s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 2.2 task/s, elapsed: 1923s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=0.63s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=73.71s).
Accumulating evaluation results...
DONE (t=9.67s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.360
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.568
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.387
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.145
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.339
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.487
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.438
 Average Recall     (AR) @[ IoU=0.50:0.9

In [ ]:
!zip -r detected_dcp.zip out_dcp_coco/
!cp -r detected_dcp.zip /content/drive/MyDrive/hazy_detect

  adding: out_dcp_coco/ (stored 0%)
  adding: out_dcp_coco/JPEGImages/ (stored 0%)
  adding: out_dcp_coco/JPEGImages/SH_Google_516.png (deflated 4%)
  adding: out_dcp_coco/JPEGImages/XG_Baidu_511.png (deflated 3%)
  adding: out_dcp_coco/JPEGImages/QD_Bing_109.png (deflated 4%)
  adding: out_dcp_coco/JPEGImages/BL_Google_463.png (deflated 3%)
  adding: out_dcp_coco/JPEGImages/BJ_Bing_082.png (deflated 2%)
  adding: out_dcp_coco/JPEGImages/CD_Baidu_453.png (deflated 2%)
  adding: out_dcp_coco/JPEGImages/LSJ_Baidu_351.png (deflated 3%)
  adding: out_dcp_coco/JPEGImages/XR_Bing_756.png (deflated 3%)
  adding: out_dcp_coco/JPEGImages/YD_Bing_429.png (deflated 2%)
  adding: out_dcp_coco/JPEGImages/TS_Baidu_422.png (deflated 4%)
  adding: out_dcp_coco/JPEGImages/TJ_Baidu_487.png (deflated 2%)
  adding: out_dcp_coco/JPEGImages/GSGL_Baidu_036.png (deflated 2%)
  adding: out_dcp_coco/JPEGImages/SJZ_Baidu_099.png (deflated 5%)
  adding: out_dcp_coco/JPEGImages/GSGL_Baidu_870.png (deflated 3%)
  a

### Refined

In [ ]:
!cp /content/drive/MyDrive/new_test.json out_refined

In [ ]:
!ls out_refined/JPEGImages

AM_Bing_211.png        HazyDr_Google_188.png  MLS_Bing_034.png
AM_Bing_217.png        HazyDr_Google_240.png  MLS_Bing_036.png
AM_Bing_222.png        HazyDr_Google_277.png  MLS_Bing_046.png
AM_Bing_229.png        HazyDr_Google_289.png  MLS_Bing_068.png
AM_Bing_232.png        HazyDr_Google_298.png  MLS_Bing_085.png
AM_Bing_242.png        HazyDr_Google_324.png  MLS_Bing_095.png
AM_Bing_243.png        HazyDr_Google_359.png  MLS_Bing_108.png
AM_Bing_274.png        HazyDr_Google_384.png  MLS_Bing_115.png
AM_Bing_318.png        HazyDr_Google_475.png  MLS_Bing_118.png
AM_Bing_422.png        HazyDr_Google_476.png  MLS_Bing_123.png
AM_Bing_464.png        HazyDr_Google_602.png  MLS_Bing_130.png
AM_Bing_536.png        HazyDr_Google_622.png  MLS_Bing_132.png
AM_Bing_640.png        HazyDr_Google_647.png  MLS_Bing_141.png
AM_Bing_727.png        HazyDr_Google_678.png  MLS_Bing_168.png
AM_Google_019.png      HazyDr_Google_753.png  MLS_Bing_174.png
AM_Google_035.png      HazyDr_Google_798.png  MLS_Bing_

In [ ]:
method = 'out_refined'
!rm -rf out_out_refined
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, True, f'./out_{method}', 0.3)
metric = dataset.evaluate(outputs, metric='bbox')

loading annotations into memory...
Done (t=0.17s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 2.3 task/s, elapsed: 1841s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=0.84s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=67.58s).
Accumulating evaluation results...
DONE (t=8.74s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.359
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.571
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.383
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.145
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.342
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.486
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.442
 Average Recall     (AR) @[ IoU=0.50:0.9

In [34]:
!zip -r detected_refined.zip out_out_refined/
!cp -r detected_refined.zip /content/drive/MyDrive/hazy_detect

  adding: out_out_refined/ (stored 0%)
  adding: out_out_refined/JPEGImages/ (stored 0%)
  adding: out_out_refined/JPEGImages/SH_Google_516.png (deflated 6%)
  adding: out_out_refined/JPEGImages/XG_Baidu_511.png (deflated 8%)
  adding: out_out_refined/JPEGImages/QD_Bing_109.png (deflated 5%)
  adding: out_out_refined/JPEGImages/BL_Google_463.png (deflated 6%)
  adding: out_out_refined/JPEGImages/BJ_Bing_082.png (deflated 4%)
  adding: out_out_refined/JPEGImages/CD_Baidu_453.png (deflated 5%)
  adding: out_out_refined/JPEGImages/LSJ_Baidu_351.png (deflated 6%)
  adding: out_out_refined/JPEGImages/XR_Bing_756.png (deflated 4%)
  adding: out_out_refined/JPEGImages/YD_Bing_429.png (deflated 8%)
  adding: out_out_refined/JPEGImages/TS_Baidu_422.png (deflated 5%)
  adding: out_out_refined/JPEGImages/TJ_Baidu_487.png (deflated 5%)
  adding: out_out_refined/JPEGImages/GSGL_Baidu_036.png (deflated 6%)
  adding: out_out_refined/JPEGImages/SJZ_Baidu_099.png (deflated 7%)
  adding: out_out_refined

# Dehazenet

In [35]:
method = 'dehazenet_coco'
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, True, f'./out_{method}', 0.3)
metric = dataset.evaluate(outputs, metric='bbox')

loading annotations into memory...
Done (t=0.26s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 2.3 task/s, elapsed: 1877s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=0.90s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=61.73s).
Accumulating evaluation results...
DONE (t=9.06s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.356
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.570
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.381
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.143
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.338
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.482
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.436
 Average Recall     (AR) @[ IoU=0.50:0.9

In [36]:
!zip -r detected_dehazenet.zip out_dehazenet/
!cp -r detected_dehazenet.zip /content/drive/MyDrive/hazy_detect

	zip warning: name not matched: out_dehazenet/

zip error: Nothing to do! (try: zip -r detected_dehazenet.zip . -i out_dehazenet/)
cp: cannot stat 'detected_dehazenet.zip': No such file or directory


# IDE

In [38]:
method = 'dide_coco'
!cp /content/drive/MyDrive/new_test.json dide_coco
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, True, f'./out_{method}', 0.3)
metric = dataset.evaluate(outputs, metric='bbox')

mkdir: cannot create directory ‘out_ide_coco’: File exists
loading annotations into memory...


NotADirectoryError: ignored

In [ ]:
!zip -r detected_ide.zip out_ide_coco/
!cp -r detected_ide.zip /content/drive/MyDrive/hazy_detect

# RTTS

In [39]:
method = 'rtts_coco'
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, True, f'./out_{method}', 0.3)
metric = dataset.evaluate(outputs, metric='bbox')

loading annotations into memory...
Done (t=0.26s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 2.3 task/s, elapsed: 1876s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=8.58s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=60.96s).
Accumulating evaluation results...
DONE (t=8.75s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.359
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.571
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.383
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.145
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.342
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.486
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.442
 Average Recall     (AR) @[ IoU=0.50:0.9

In [40]:
!zip -r rtts_coco.zip out_rtts_coco/
!cp -r rtts_coco.zip /content/drive/MyDrive/hazy_detect

  adding: out_rtts_coco/ (stored 0%)
  adding: out_rtts_coco/JPEGImages/ (stored 0%)
  adding: out_rtts_coco/JPEGImages/SH_Google_516.png (deflated 6%)
  adding: out_rtts_coco/JPEGImages/XG_Baidu_511.png (deflated 8%)
  adding: out_rtts_coco/JPEGImages/QD_Bing_109.png (deflated 5%)
  adding: out_rtts_coco/JPEGImages/BL_Google_463.png (deflated 6%)
  adding: out_rtts_coco/JPEGImages/BJ_Bing_082.png (deflated 4%)
  adding: out_rtts_coco/JPEGImages/CD_Baidu_453.png (deflated 5%)
  adding: out_rtts_coco/JPEGImages/LSJ_Baidu_351.png (deflated 6%)
  adding: out_rtts_coco/JPEGImages/XR_Bing_756.png (deflated 4%)
  adding: out_rtts_coco/JPEGImages/YD_Bing_429.png (deflated 8%)
  adding: out_rtts_coco/JPEGImages/TS_Baidu_422.png (deflated 5%)
  adding: out_rtts_coco/JPEGImages/TJ_Baidu_487.png (deflated 5%)
  adding: out_rtts_coco/JPEGImages/GSGL_Baidu_036.png (deflated 6%)
  adding: out_rtts_coco/JPEGImages/SJZ_Baidu_099.png (deflated 7%)
  adding: out_rtts_coco/JPEGImages/GSGL_Baidu_870.png (